In [15]:
using Pkg
Pkg.activate("../Project.toml")

using ITensors
import NDTensors
include("summary.jl")
include("utils.jl")

  Activating project at `~/Documents/QML Project/QuantumInspiredML`


loadMPS (generic function with 2 methods)

In [16]:
function generate_startingMPS(χ_init, site_indices::Vector{Index{Int64}};
    num_classes = 2, random_state=nothing)
    """Generate the starting weight MPS, W using values sampled from a 
    Gaussian (normal) distribution. Accepts a χ_init parameter which
    specifies the initial (uniform) bond dimension of the MPS."""
    
    if random_state !== nothing
        # use seed if specified
        Random.seed!(random_state)
        println("Generating initial weight MPS with bond dimension χ = $χ_init
        using random state $random_state.")
    else
        println("Generating initial weight MPS with bond dimension χ = $χ_init.")
    end

    W = randomMPS(ComplexF64,site_indices, linkdims=χ_init)

    label_idx = Index(num_classes, "f(x)")

    # get the site of interest and copy over the indices at the last site where we attach the label 
    old_site_idxs = inds(W[end])
    new_site_idxs = old_site_idxs, label_idx
    new_site = randomITensor(new_site_idxs)

    # add the new site back into the MPS
    W[end] = new_site

    # normalise the MPS
    normalize!(W)

    # canonicalise - bring MPS into canonical form by making all tensors 1,...,j-1 left orthogonal
    # here we assume we start at the right most index
    last_site = length(site_indices)
    orthogonalize!(W, last_site)

    return W

end


generate_startingMPS (generic function with 1 method)

In [17]:
random_state=123456
update_iters = 9
χ_init = 4
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_splits_txt("datasets/ECG_train.txt", 
    "datasets/ECG_val.txt", "datasets/ECG_test.txt")

X_train = vcat(X_train, X_val)
y_train = vcat(y_train, y_val)   
   
# first, create the site indices for the MPS and product states 
num_mps_sites = size(X_train)[2]
sites = siteinds("S=1/2", num_mps_sites)
println("Using χ_init=$χ_init and a maximum of $nsweep sweeps...")
println("Using $update_iters iterations per update.")

# now let's handle the training/validation/testing data
# rescale using a robust sigmoid transform
scaler = fit_scaler(RobustSigmoidTransform, X_train; positive=true);
X_train_scaled = transform_data(scaler, X_train)
X_val_scaled = transform_data(scaler, X_val)
X_test_scaled = transform_data(scaler, X_test)

# generate product states using rescaled data

training_states = generate_all_product_states(X_train_scaled, y_train, "train", sites)
validation_states = generate_all_product_states(X_val_scaled, y_val, "valid", sites)
testing_states = generate_all_product_states(X_test_scaled, y_test, "test", sites)

# generate the starting MPS with unfirom bond dimension χ_init and random values (with seed if provided)
num_classes = length(unique(y_train))
W = generate_startingMPS(χ_init, sites; num_classes=num_classes, random_state=random_state)


    ;

Using χ_init=4 and a maximum of nsweep sweeps...
Using 9 iterations per update.
Initialising train states.
Initialising valid states.
Initialising test states.
Generating initial weight MPS with bond dimension χ = 4
        using random state 123456.


In [18]:
saveMPS(W, "saved/test")

Succesfully saved mps W at saved/test.h5


In [20]:
W2 = loadMPS("saved/test", "W");

In [21]:
isapprox(W,W2)

true

ITensor ord=2 (dim=2|id=723|"S=1/2,Site,n=1") (dim=2|id=813|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}